# Deep Neural Net for Sentiment Classification

This Deep Net performs sentiment analysis on the IMDB review dataset.

In [ ]:
import os

from tensorflow.keras.datasets import imdb
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout, Activation
from tensorflow.keras.layers import Embedding
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, TensorBoard

from tensorflow.keras.models import load_model

import sklearn.metrics
from sklearn.metrics import roc_auc_score

import pandas as pd

import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline

#### Set Hyperparameters

In [ ]:
output_dir = 'model_output/deep-net'

epochs = 3
batch_size = 64
patience = 10
val_split = .3

n_dim = 128
n_unique_words = 10000
max_review_length = 200
pad_type = trunc_type = 'pre'

n_dense = 64
dropout = .5

#### Load Data

In [ ]:
(X_train, y_train), (X_valid, y_valid) = imdb.load_data(num_words=n_unique_words)

In [ ]:
# X_valid[0]

In [ ]:
for x in X_train[0:6]:
    print(len(x))

In [ ]:
y_train[0:6]

In [ ]:
len(X_train), len(y_train)

#### Restore Words from index

In [ ]:
word_index = imdb.get_word_index()
word_index = {k : (v+3) for k,v in word_index.items()}
word_index['PAD'] = 0
word_index['START'] = 1
word_index['UNK'] = 2

In [ ]:
# word_index

In [ ]:
index_word = {v:k for k,v in word_index.items()}

In [ ]:
# index_word

In [ ]:
X_train[0]

In [ ]:
' '.join(index_word[id] for id in X_train[0])

In [ ]:
(all_X_train, _), (all_X_valid, _) = imdb.load_data()

In [ ]:
' '.join(index_word[id] for id in all_X_train[0])

#### Preprocess Data

In [ ]:
X_train = pad_sequences(X_train, maxlen=max_review_length, padding=pad_type, truncating=trunc_type, value=0)
X_valid = pad_sequences(X_valid, maxlen=max_review_length, padding=pad_type, truncating=trunc_type, value=0)

In [ ]:
X_valid[0]

In [ ]:
for x in X_train[0:6]:
    print(len(x))

In [ ]:
' '.join(index_word[id] for id in X_train[0])

In [ ]:
' '.join(index_word[id] for id in X_train[5])

#### Design Deep Net Architecture

In [ ]:
model = Sequential()
model.add(Embedding(n_unique_words, n_dim, input_length=max_review_length))
model.add(Flatten())
model.add(Dense(n_dense, activation='relu'))

model.add(Dense(n_dense, activation='relu'))
model.add(Dropout(dropout))

model.add(Dense(1, activation='sigmoid'))

In [ ]:
model.summary()

In [ ]:
# Embedding 1
n_dim, n_unique_words, n_dim*n_unique_words

In [ ]:
# Flatten 1
max_review_length, n_dim, n_dim*max_review_length

In [ ]:
# Dense 1 (Fully Connected)
n_dense, n_dim*max_review_length*n_dense + n_dense

In [ ]:
# Dense 2
n_dense + 1

#### Configure the Model

In [ ]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
modelCheckpoint = ModelCheckpoint(monitor='val_accuracy', filepath=output_dir + '/imdb-deep-net.hdf5', save_best_only=True, mode='max')
earlyStopping = EarlyStopping(monitor='val_accuracy', mode='max', patience=patience)

if not os.path.exists(output_dir):
    os.makedirs(output_dir)

#### TensorBoard

In [ ]:
tensorboard = TensorBoard("../logs/imdb-deep-net")

### Train the Model

In [ ]:
history = model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, verbose=1,
                    validation_split=val_split, callbacks=[modelCheckpoint, earlyStopping, tensorboard])

#### Evaluate

In [ ]:
model = load_model(output_dir + '/imdb-deep-net.hdf5')

In [ ]:
y_hat = model.predict_proba(X_valid)
final_loss, final_acc = model.evaluate(X_valid, y_valid, verbose = 2)
print("Final loss: {0:.4f}, final accuracy: {1:.4f}".format(final_loss, final_acc))

In [ ]:
plt.hist(y_hat)
_ = plt.axvline(x=0.5, color='orange')

In [ ]:
pct_auc = roc_auc_score(y_valid, y_hat) * 100

In [ ]:
print('{:0.2f}'.format(pct_auc))
print(np.std(history.history['loss']))

In [ ]:
fpr, tpr, _ = sklearn.metrics.roc_curve(y_valid, y_hat)
roc_auc = sklearn.metrics.auc(fpr, tpr)

plt.figure()
lw = 2
plt.plot(fpr, tpr, color='darkorange',
         lw=lw, label='ROC curve (area = %0.2f)' % roc_auc)

plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic')
plt.legend(loc="lower right")

plt.show()

In [ ]:
ydf = pd.DataFrame(list(zip(y_hat[:,0], y_valid)), columns=['y_hat', 'y'])
ydf.head(10)

In [ ]:
' '.join(index_word[id] for id in all_X_valid[0])

In [ ]:
' '.join(index_word[id] for id in all_X_valid[6])

In [ ]:
ydf[(ydf.y == 0) & (ydf.y_hat > 0.9)].head(10)

In [ ]:
' '.join(index_word[id] for id in all_X_valid[112])

In [ ]:
ydf[(ydf.y == 1) & (ydf.y_hat < 0.1)].head(10)

In [ ]:
' '.join(index_word[id] for id in all_X_valid[101])

In [ ]:
import nltk
from nltk.tokenize import word_tokenize, sent_tokenize

imdb_review = """This is a wolverine movie aimed at adults and definitely not for small children. The story is set in the future and Logan is taking care of Charles Xavier who has grown old and is incapable of taking care of himself. Logan, himself is now older and his health is slowly deteriorating as well. A young girl mutant enters their lives which sets off a story, which is not about super heroes fighting bad guys, but one that is on a more human level. "Logan", is a beautifully made X-Men film that deals with themes of importance of family, loyalty, and ultimately how Logan himself deals with new found emotions which he is experiencing for the very first time in his life.

This is the crowning achievement in Hugh Jackman's role as Wolverine. It is the movie he and director, James Mangold had always wanted to make. It is a dramatic work of art and for X-Men fans, has plenty of action and special FX which are a definite cut above anything that has been seen before in Wolverine/X-Men films.

A MUST SEE FOR ANY FAN AND A GOOD ENOUGH FILM TO STAND ALONE FOR ANYONE!!"""
review_tokens = word_tokenize(imdb_review)

In [ ]:
review_index = [v if v <= 10000 else 3 for k,v in word_index.items() if k in review_tokens]
review_index = np.array([review_index], dtype='int32')
review_index.reshape((len(review_index[0])))

In [ ]:
review_index.shape

In [ ]:
review_index = pad_sequences(review_index, maxlen=max_review_length, padding=pad_type, truncating=trunc_type, value=0)

In [ ]:
review_index
review_index.shape

In [ ]:
y_hat = model.predict_proba(review_index)

In [ ]:
y_hat